# Model Assessment Notebook

Used on the inter-intra notebook, to get fair assessment between notebooks as train/validate/splits may be different between users due to different approaches and unset random seeds.

In [2]:
!zip -r oct_unet_modified_20230206-125331.zip ./oct_unet_modified_20230206-125331

  adding: oct_unet_modified_20230206-125331/ (stored 0%)
  adding: oct_unet_modified_20230206-125331/assets/ (stored 0%)
  adding: oct_unet_modified_20230206-125331/variables/ (stored 0%)
  adding: oct_unet_modified_20230206-125331/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: oct_unet_modified_20230206-125331/variables/variables.index (deflated 79%)
  adding: oct_unet_modified_20230206-125331/saved_model.pb (deflated 91%)
  adding: oct_unet_modified_20230206-125331/keras_metadata.pb (deflated 95%)


In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from tensorflow import keras
from tensorflow.keras.utils import load_img

from sklearn.metrics import confusion_matrix, classification_report

import utils

In [ ]:
IMG_WIDTH_ORIGINAL=512
IMG_HEIGHT_ORIGINAL=1024
IMG_WIDTH = 256
IMG_HEIGHT = 512
IMG_CHANNELS = 1 # grayscale images
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH)
NUM_CLASSES = 8 #8 in case just fluids is 4, in case fluids and layers is 8
BATCH_SIZE = 4 # try 4, 8, 12, 16, 32

SRF_CLASS = 6
IRF_CLASS = 7

CLASS_LABELS = (
    "Above ILM",
    "ILM-IPL/INL",
    "IPL/INL-RPE",
    "RPE-BM",
    "Under BM",
    "PED",
    "SRF",
    "IRF",
)

In [ ]:
class AROISequenceNoTransform(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_train_paths = input_img_paths
        self.target_train_paths = target_img_paths

    def __len__(self):
        return math.ceil(len(self.target_train_paths) / self.batch_size)

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_train_paths = self.input_train_paths[i : i + self.batch_size]
        batch_target_train_paths = self.target_train_paths[i : i + self.batch_size]
        x = np.zeros((BATCH_SIZE,) + self.img_size + (1,), dtype="float32")
        for j, path in enumerate(batch_input_train_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            img=np.reshape(img, (IMG_HEIGHT, IMG_WIDTH, 1))
            x[j] = img/.255
        y = np.zeros((BATCH_SIZE,) + self.img_size + (1,), dtype="uint8")
        for j, path in enumerate(batch_target_train_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            img = np.reshape(img, (IMG_HEIGHT, IMG_WIDTH, 1))
            y[j] = img
        return x, y



In [ ]:
model = keras.models.load_model("./oct_model_20230125-092315", compile=False)

In [ ]:
INTER_INTRA_DATA = Path("./data/AROI_labelled_scans/inter_intra/")
inter_intra_mask_path = sorted([file for file in INTER_INTRA_DATA.glob("**/number/*png")])
inter_intra_raw_path = sorted([file for file in INTER_INTRA_DATA.glob("**/raw/*png")])

inter_intra_seq = AROISequenceNoTransform(BATCH_SIZE, IMG_SIZE, inter_intra_raw_path, inter_intra_mask_path)

inter_predictions = model.predict(inter_intra_seq)
print(inter_predictions.shape)
inter_predictions = np.argmax(inter_predictions, axis=-1)

inter_masks = utils.get_masks_from_keras_sequence(inter_intra_seq)

cf_matrix = confusion_matrix(inter_masks.flatten(), inter_predictions.flatten())
cf_matrix_normal = cf_matrix.astype('float32') / cf_matrix.sum(axis=1)[:, np.newaxis]

utils.make_confusion_matrix(cf_matrix_normal, count = False, percent = False, figsize=(7,7), cbar=True, cmap='plasma')

In [ ]:
print(f"{' InterIntra Report ':=^55}")
print(classification_report(inter_masks.flatten(), inter_predictions.flatten(), target_names=CLASS_LABELS))